# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Arguments" data-toc-modified-id="Arguments-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Arguments</a></div><div class="lev1 toc-item"><a href="#MetaCyc-subset-of-QM9" data-toc-modified-id="MetaCyc-subset-of-QM9-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>MetaCyc subset of QM9</a></div><div class="lev1 toc-item"><a href="#Define-model-and-optimizer" data-toc-modified-id="Define-model-and-optimizer-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define model and optimizer</a></div><div class="lev1 toc-item"><a href="#Statistics" data-toc-modified-id="Statistics-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Statistics</a></div><div class="lev1 toc-item"><a href="#Create-model" data-toc-modified-id="Create-model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Create model</a></div><div class="lev1 toc-item"><a href="#get-the-best-checkpoint-if-available-without-training" data-toc-modified-id="get-the-best-checkpoint-if-available-without-training-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>get the best checkpoint if available without training</a></div>

In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable

import time
import argparse
import os
import numpy as np
import sys
sys.path.append(os.getcwd())
# Our Modules
import utils
from utils import datasets
from models.MPNN import MPNN
from models.MPNNv2 import MPNNv2
from models.MPNNv3 import MPNNv3
from utils.LogMetric import AverageMeter, Logger


# Arguments

In [2]:
parser = argparse.ArgumentParser(description='Neural message passing')

parser.add_argument('--dataset', default='qm9', help='QM9')
parser.add_argument('--edge-rep', default='raw_distance', choices=['raw_distance','chem_graph','distance_bin'] )
parser.add_argument('--datasetPath', default='./mpnn-data/qm9/dsgdb9nsd/', help='dataset path')
parser.add_argument('--logPath', default='./log/qm9/mpnn/', help='log path')
parser.add_argument('--plotLr', default=False, help='allow plotting the data')
parser.add_argument('--plotPath', default='./plot/qm9/mpnn/', help='plot path')
parser.add_argument('--resume', default='./checkpoint/qm9/mpnn/',
                    help='path to latest checkpoint')
# Optimization Options
parser.add_argument('--batch-size', type=int, default=100, metavar='N',
                    help='Input batch size for training (default: 20)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Enables CUDA training')
parser.add_argument('--epochs', type=int, default=50, metavar='N',
                    help='Number of epochs to train (default: 50)')
parser.add_argument('--lr', type=lambda x: restricted_float(x, [1e-5, 1e-2]), default=1e-3, metavar='LR',
                    help='Initial learning rate [1e-5, 5e-4] (default: 1e-4)')
parser.add_argument('--lr-decay', type=lambda x: restricted_float(x, [.01, 1]), default=0.6, metavar='LR-DECAY',
                    help='Learning rate decay factor [.01, 1] (default: 0.6)')
parser.add_argument('--schedule', type=list, default=[0.1, 0.9], metavar='S',
                    help='Percentage of epochs to start the learning rate decay [0, 1] (default: [0.1, 0.9])')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                    help='SGD momentum (default: 0.9)')
# i/o
parser.add_argument('--log-interval', type=int, default=20, metavar='N',
                    help='How many batches to wait before logging training status')
# Accelerating
parser.add_argument('--prefetch', type=int, default=2, help='Pre-fetching threads.')

# Model modification
parser.add_argument('--model', type=str,help='MPNN model name [MPNN, MPNNv2, MPNNv3]',
                        default='MPNN')


args = parser.parse_args(['--model','MPNNv2', '--edge-rep','chem_graph', ])#'--no-cuda'])
print(args)


Namespace(batch_size=100, dataset='qm9', datasetPath='./mpnn-data/qm9/dsgdb9nsd/', edge_rep='chem_graph', epochs=50, logPath='./log/qm9/mpnn/', log_interval=20, lr=0.001, lr_decay=0.6, model='MPNNv2', momentum=0.9, no_cuda=False, plotLr=False, plotPath='./plot/qm9/mpnn/', prefetch=2, resume='./checkpoint/qm9/mpnn/', schedule=[0.1, 0.9])


In [4]:
def restricted_float(x, inter):
    x = float(x)
    if x < inter[0] or x > inter[1]:
        raise argparse.ArgumentTypeError("%r not in range [1e-5, 1e-4]"%(x,))
    return x


In [5]:

best_er1 = 0

    # Check if CUDA is enabled
args.cuda = not args.no_cuda and torch.cuda.is_available()

    # Load data
root = args.datasetPath
root

'./mpnn-data/qm9/dsgdb9nsd/'

# MetaCyc subset of QM9

In [8]:
print('Prepare files')
import pandas as pd
qm9_metacyc = pd.read_table('Dataset/QM9_metacyc.tab')
qm9_metacyc

Prepare files


,FrameId,Name,Kegg,InChI,Smiles,Gibbs-0,cpd,nC
0,BUTYRAMIDE,butyramide,NIL,"InChI=1S/C4H9NO/c1-2-3-4(5)6/h2-3H2,1H3,(H2,5,6)",CCCC(N)=O,51.370094,./dsgdb9nsd_000262.xyz,4
1,CPD-282,cyclohex-2-enone,C02395,"InChI=1S/C6H8O/c7-6-4-2-1-3-5-6/h2,4H,1,3,5H2",C1(CCC(C=C1)=O),76.978966,./dsgdb9nsd_002060.xyz,6
2,CPD-7031,3-methylbutanal,C07329,"InChI=1S/C5H10O/c1-5(2)3-4-6/h4-5H,3H2,1-2H3",CC(C)C[CH]=O,71.711205,./dsgdb9nsd_000255.xyz,5
3,CPD-9383,glycine methyl ester,NIL,"InChI=1S/C3H7NO2/c1-6-3(5)2-4/h2,4H2,1H3",C([NH2])C(=O)OC,-3.392161,./dsgdb9nsd_000365.xyz,3
4,CYTOSINE,cytosine,C00380,"InChI=1S/C4H5N3O/c5-3-1-2-6-4(8)7-3/h1-2H,(H3,...",C1(NC(=O)N=C(N)C=1),19.905600,./dsgdb9nsd_004243.xyz,4
5,CYTOSINE,cytosine,C00380,"InChI=1S/C4H5N3O/c5-3-1-2-6-4(8)7-3/h1-2H,(H3,...",C1(NC(=O)N=C(N)C=1),19.905600,./dsgdb9nsd_004318.xyz,4
6,CYTOSINE,cytosine,C00380,"InChI=1S/C4H5N3O/c5-3-1-2-6-4(8)7-3/h1-2H,(H3,...",C1(NC(=O)N=C(N)C=1),19.905600,./dsgdb9nsd_004371.xyz,4
7,CYTOSINE,cytosine,C00380,"InChI=1S/C4H5N3O/c5-3-1-2-6-4(8)7-3/h1-2H,(H3,...",C1(NC(=O)N=C(N)C=1),19.905600,./dsgdb9nsd_004403.xyz,4
8,URACIL,uracil,C00106,"InChI=1S/C4H4N2O2/c7-3-1-2-5-4(8)6-3/h1-2H,(H2...",C1(=CC(NC(=O)N1)=O),-28.995518,./dsgdb9nsd_004258.xyz,4
9,URACIL,uracil,C00106,"InChI=1S/C4H4N2O2/c7-3-1-2-5-4(8)6-3/h1-2H,(H2...",C1(=CC(NC(=O)N1)=O),-28.995518,./dsgdb9nsd_004294.xyz,4


In [9]:
files = [f for f in os.listdir(root) if os.path.isfile(os.path.join(root, f))
                                        and './' + os.path.basename(f) in 
                                             set(qm9_metacyc['cpd'].values)]

idx = np.random.permutation(len(files))
idx = idx.tolist()
files

['dsgdb9nsd_007723.xyz',
 'dsgdb9nsd_000031.xyz',
 'dsgdb9nsd_000004.xyz',
 'dsgdb9nsd_000544.xyz',
 'dsgdb9nsd_002071.xyz',
 'dsgdb9nsd_001178.xyz',
 'dsgdb9nsd_004376.xyz',
 'dsgdb9nsd_000007.xyz',
 'dsgdb9nsd_004332.xyz',
 'dsgdb9nsd_000767.xyz',
 'dsgdb9nsd_000471.xyz',
 'dsgdb9nsd_000134.xyz',
 'dsgdb9nsd_002706.xyz',
 'dsgdb9nsd_006768.xyz',
 'dsgdb9nsd_004403.xyz',
 'dsgdb9nsd_000026.xyz',
 'dsgdb9nsd_001276.xyz',
 'dsgdb9nsd_000289.xyz',
 'dsgdb9nsd_004243.xyz',
 'dsgdb9nsd_007726.xyz',
 'dsgdb9nsd_001680.xyz',
 'dsgdb9nsd_004958.xyz',
 'dsgdb9nsd_006771.xyz',
 'dsgdb9nsd_004281.xyz',
 'dsgdb9nsd_004279.xyz',
 'dsgdb9nsd_000040.xyz',
 'dsgdb9nsd_000225.xyz',
 'dsgdb9nsd_005379.xyz',
 'dsgdb9nsd_004988.xyz',
 'dsgdb9nsd_000272.xyz',
 'dsgdb9nsd_000286.xyz',
 'dsgdb9nsd_005349.xyz',
 'dsgdb9nsd_008901.xyz',
 'dsgdb9nsd_005357.xyz',
 'dsgdb9nsd_000358.xyz',
 'dsgdb9nsd_004409.xyz',
 'dsgdb9nsd_002637.xyz',
 'dsgdb9nsd_000068.xyz',
 'dsgdb9nsd_000224.xyz',
 'dsgdb9nsd_000037.xyz',


In [22]:
valid_ids = [files[i] for i in idx[0:100]]
test_ids = [files[i] for i in idx[100:200]]
train_ids = [files[i] for i in idx[200:]]

data_train = utils.Qm9(root, train_ids, edge_transform=datasets.qm9_edges, e_representation=args.edge_rep)
data_valid = utils.Qm9(root, valid_ids, edge_transform=datasets.qm9_edges, e_representation=args.edge_rep)
data_test = utils.Qm9(root, test_ids, edge_transform=datasets.qm9_edges, e_representation=args.edge_rep)

# Define model and optimizer


In [23]:
print('Define model')
# Select one graph
g_tuple, l = data_train[0]
g, h_t, e = g_tuple
dG = -2


Define model


-193.065979

In [26]:
vals = utils.datasets.get_values(data_valid, 0, len(data_valid), ['target_mean','target_std'])
param = np.array([file_res['params'] for file_res in vals])
param[:,dG]

array([-323.695493, -382.858645, -326.835041, -271.678618, -378.802114,
       -248.478935, -410.957679, -394.893227, -154.998148, -366.350902,
       -272.870294, -438.199233, -343.34119 , -169.885594,  -56.544961,
       -229.995393, -306.43831 , -361.767103, -310.974733, -438.207001,
       -265.467001, -287.538453, -410.958658, -458.043037, -394.894525,
       -359.417566, -301.417032, -398.707079, -348.0892  , -414.770508,
       -323.477888, -362.761949, -394.877301, -301.433583, -272.867062,
       -398.678207, -362.978421, -382.641387, -268.316982, -297.582689,
       -362.989148, -304.775338, -377.845832, -271.697004, -286.571234,
       -410.934076, -233.581067, -250.614891, -394.883467, -272.869568,
       -231.184401, -394.888044, -361.562566,  -93.431246, -301.424079,
       -194.28893 , -226.170637, -346.923014, -344.719856, -345.729675,
       -233.578016, -410.950403, -232.192047, -401.081561, -402.268552,
       -386.229453, -382.640636, -310.97521 , -323.47761 , -325.

# Statistics

In [12]:
print('\tStatistics')
stat_dict = datasets.get_graph_stats(data_valid, ['target_mean', 'target_std'] ) #, multiprocess_p=False)
stat_dict

	Statistics


{'target_mean': array([  2.89543100e+00,   5.46472000e+01,  -2.43896000e-01,
          9.09900000e-03,   2.52998000e-01,   7.51444309e+02,
          1.10979550e-01,  -3.18908946e+02,  -3.18902242e+02,
         -3.18901298e+02,  -3.18939408e+02,   2.38602700e+01]),
 'target_std': array([  1.95161320e+00,   1.40061087e+01,   2.52599997e-02,
          4.31003956e-02,   5.40552532e-02,   3.38578092e+02,
          3.44841183e-02,   8.07238264e+01,   8.07228018e+01,
          8.07228018e+01,   8.07261425e+01,   6.13787596e+00])}

In [13]:
data_train.set_target_transform(lambda x: datasets.normalize_data(x,stat_dict['target_mean'],
                                                                            stat_dict['target_std']))
data_valid.set_target_transform(lambda x: datasets.normalize_data(x, stat_dict['target_mean'],
                                                                            stat_dict['target_std']))
data_test.set_target_transform(lambda x: datasets.normalize_data(x, stat_dict['target_mean'],
                                                                           stat_dict['target_std']))

In [14]:
    train_loader = torch.utils.data.DataLoader(data_train,
                                               batch_size=args.batch_size, shuffle=True,
                                               collate_fn=datasets.collate_g,
                                               num_workers=args.prefetch, pin_memory=True)
    valid_loader = torch.utils.data.DataLoader(data_valid,
                                               batch_size=args.batch_size, collate_fn=datasets.collate_g,
                                               num_workers=args.prefetch, pin_memory=True)
    test_loader = torch.utils.data.DataLoader(data_test,
                                              batch_size=args.batch_size, collate_fn=datasets.collate_g,
                                              num_workers=args.prefetch, pin_memory=True)


# Create model

In [15]:
    in_n = [len(h_t[0]), len(list(e.values())[0])]
    hidden_state_size = 73
    message_size = 73
    n_layers = 3
    l_target = len(l)
    type ='regression'
    if args.model == 'MPNNv2':
        model = MPNNv2(in_n, [5, 15, 15], [10, 20, 20], l_target, type=type)
    elif args.model == 'MPNNv3':
        model = MPNNv3([1, 2, 3, 4], in_n, [5, 15, 15], 30, l_target, type=type)
    else:
        model = MPNN(in_n, hidden_state_size, message_size, n_layers, l_target, type=type)
    del in_n, hidden_state_size, message_size, n_layers, l_target, type

    print('Optimizer')
    optimizer = optim.Adam(model.parameters(), lr=args.lr)

    criterion = nn.MSELoss()

    evaluation = lambda output, target: torch.mean(torch.abs(output - target) / torch.abs(target))

    print('Logger')
    logger = Logger(args.logPath)

    lr_step = (args.lr-args.lr*args.lr_decay)/(args.epochs*args.schedule[1] - args.epochs*args.schedule[0])


Optimizer
Logger


# get the best checkpoint if available without training

In [16]:
if args.resume:
        checkpoint_dir = args.resume
        best_model_file = os.path.join(checkpoint_dir, 'model_best.pth')
        if not os.path.isdir(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        if os.path.isfile(best_model_file):
            print("=> loading best model '{}'".format(best_model_file))
            checkpoint = torch.load(best_model_file)
            args.start_epoch = checkpoint['epoch']
            best_acc1 = checkpoint['best_er1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded best model '{}' (epoch {})".format(best_model_file, checkpoint['epoch']))
        else:
            print("=> no best model found at '{}'".format(best_model_file))


=> loading best model './checkpoint/qm9/mpnn/model_best.pth'
=> loaded best model './checkpoint/qm9/mpnn/model_best.pth' (epoch 50)


# Check CUDA

In [17]:
if args.cuda:
        print('\t* Cuda')
        model = model.cuda()
        criterion = criterion.cuda()


	* Cuda


# For testing

In [18]:
def validate(val_loader, model, criterion, evaluation, logger=None):
    batch_time = AverageMeter()
    losses = AverageMeter()
    error_ratio = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (g, h, e, target) in enumerate(val_loader):

        # Prepare input data
        if args.cuda:
            g, h, e, target = g.cuda(), h.cuda(), e.cuda(), target.cuda()
        g, h, e, target = Variable(g), Variable(h), Variable(e), Variable(target)

        # Compute output
        output = model(g, h, e)

        # Logs
        losses.update(criterion(output, target).data[0], g.size(0))
        error_ratio.update(evaluation(output, target).data[0], g.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.log_interval == 0 and i > 0:
            
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Error Ratio {err.val:.4f} ({err.avg:.4f})'
                  .format(i, len(val_loader), batch_time=batch_time,
                          loss=losses, err=error_ratio))

    print(' * Average Error Ratio {err.avg:.3f}; Average Loss {loss.avg:.3f}'
          .format(err=error_ratio, loss=losses))

    if logger is not None:
        logger.log_value('test_epoch_loss', losses.avg)
        logger.log_value('test_epoch_error_ratio', error_ratio.avg)

    return error_ratio.avg

validate(test_loader, model, criterion, evaluation)

 * Average Error Ratio 10.339; Average Loss 3.640


10.338644027709961

In [19]:
!cat chem-graph-slurm.out

Namespace(batch_size=100, dataset='qm9', datasetPath='/scratch/zuck016/mpnn-data/qm9/dsgdb9nsd/', epochs=50, logPath='./log/qm9/mpnn/', log_interval=20, lr=0.001, lr_decay=0.6, model='MPNNv2', momentum=0.9, no_cuda=False, plotLr=False, plotPath='./plot/qm9/mpnn/', prefetch=2, resume='./checkpoint/qm9/mpnn/', schedule=[0.1, 0.9])
Prepare files
Define model
	Statistics
	Create model
Optimizer
Logger
=> no best model found at './checkpoint/qm9/mpnn/model_best.pth'
Check cuda
	* Cuda
Epoch: [0][20/1139]	Time 1.970 (1.069)	Data 1.613 (0.836)	Loss 0.8155 (0.9853)	Error Ratio 1.5101 (1.3005)
Epoch: [0][40/1139]	Time 1.804 (1.039)	Data 1.613 (0.825)	Loss 0.8671 (0.9265)	Error Ratio 2.7260 (1.7734)
Epoch: [0][60/1139]	Time 1.727 (1.029)	Data 1.537 (0.820)	Loss 0.7746 (0.8720)	Error Ratio 2.0390 (2.0867)
Epoch: [0][80/1139]	Time 1.471 (1.025)	Data 1.286 (0.818)	Loss 0.6954 (0.8310)	Error Ratio 5.0259 (2.3057)
Epoch: [0][100/1139]	Time 1.581 (1.019)	Data 1.385 (0.815)	Loss 0.6948 (0.8100)	Error R

Epoch: [48][560/1139]	Time 2.031 (1.067)	Data 1.787 (0.866)	Loss 0.0335 (0.0327)	Error Ratio 0.5520 (0.8361)
Epoch: [48][580/1139]	Time 1.876 (1.067)	Data 1.689 (0.866)	Loss 0.0630 (0.0331)	Error Ratio 0.5067 (0.8386)
Epoch: [48][600/1139]	Time 1.971 (1.066)	Data 1.759 (0.866)	Loss 0.0317 (0.0331)	Error Ratio 0.5216 (0.8368)
Epoch: [48][620/1139]	Time 1.748 (1.066)	Data 1.545 (0.865)	Loss 0.0258 (0.0332)	Error Ratio 4.6942 (0.8401)
Epoch: [48][640/1139]	Time 1.890 (1.066)	Data 1.679 (0.865)	Loss 0.0460 (0.0333)	Error Ratio 0.5356 (0.8361)
Epoch: [48][660/1139]	Time 1.316 (1.065)	Data 1.145 (0.864)	Loss 0.0450 (0.0335)	Error Ratio 0.4999 (0.8369)
Epoch: [48][680/1139]	Time 1.377 (1.065)	Data 1.199 (0.865)	Loss 0.0306 (0.0335)	Error Ratio 0.6491 (0.8370)
Epoch: [48][700/1139]	Time 1.431 (1.065)	Data 1.221 (0.865)	Loss 0.0324 (0.0336)	Error Ratio 1.2053 (0.8377)
Epoch: [48][720/1139]	Time 1.162 (1.064)	Data 0.974 (0.864)	Loss 0.0275 (0.0336)	Error Ratio 0.4283 (0.8365)
Epoch: [48][740/113